# Step by step (Resumen)

- Lanzamos cluster en aws
- Nos conectamos por consola y ssh al cluster desde el pc personal
- Instalamos jupyterhub y clonamos el  [repo](https://github.com/camilaMejia/trabajoFinal) del proyecto con este notebook listo. (En el github hay un archivo que se llama launch.txt donde están todas las intrucciones que se lanzan por comando)
- Instalamos y cargamos todas las librerias necesarias.
- Nos traemos el .dat y el .csv desde S3 al almacenamiento local
- Creamos el indice invertido usando metapy
- Hacemos querying usando BM25
- Se hace un LDA con todos las noticias (solo content + title)
- Para cada noticia hacemos vemos cual es el topico dominante
- Asignación de la polaridad de cada noticia utilizando textBlob.



# Procedimiento en AWS

Uno de los objetivos de este trabajo estaba asociado a la posibilidad de correr todo esto en maquinas virtuales y clusters de AWS. Para ello lo que hicimos fue la creación de un cluster a traves de EMR y luego hacer un ambiente particular para instalar metapy.

Una dificultad es que estos clusters son efimeros por lo que no almacenan los notebooks, por eso lo que se hizo es que cuando lanzamos el cluster además de instalar metapy y el resto de librerias necesarias instalamos github para clonar todos los archivos necesarios menos el .dat y el csv con todas las noticias (dado que son muy pesados).

Para traer al ambiente los datos que son muy grandes lo que hacemos es copiarlos de una carpeta de aws.

Para la parte de LDA con todos los datos era necesario correr spark en el cluster (no local) por eso para esta tarea se usaron los notebooks nativos de EMR. En la versión que vemos en este notebook en realidad solo tenemos un demo co 1000 noticias para que corra suficientmeente rapido (además que el EMR consume muchos más recursos y los 50USD no son suficiente).

# Indexación y recuperación: metapy

Usando la libreria metapy lo que hacemos es simplemente configurar el archivo llamado minifig.toml en el cual especificamos en donde están los documentos que vamos a indexar (junto con su formato) para luego hacer la recuperación.

Lo que se hace en la sección de indexación es la creación del indice invertido que luego va a ser utilizado, en la sección de recuperación simplemente se crea un objeto  el cual genera un ranking (en este caso es un BM25) cuando se le pasa un query.

# LDA: spark

Lo que hacemos en este procedimiento es usar las funciones de spark para leer la información de todas las noticias para primero hacer la representación vectorial de los documentos, luego calculamos el indice invertido para finalmente hacer la reducción de dimencionalidad con LDA.

LDA lo que hace es que asigna a cada noticia un vector de k posiciones en donde la posicion i  expresa la probabilidad de que la noticia hable del topico i. Además de esto a cada noticia se le puede asignar su topico dominante.

Adicional vemos que cada topico tiene unas tokens que lo definen, dado que estos topicos no son predeterminados es necesario que se haga un analisis de qué es cada cluster con base en esos tokens representativos (algo parecido a cuando en k-means se hace un analisis de los centroides).

Algo muy importante es que en este demo es necesario crear un contexto de spark (que además es local), por el contrario en EMR el contexto ya es predeterminado.

# Sentiment Analysis

Para esta sección la idea inicial era entrenar un algoritmo sencillo, pero dado que no hay un sentimiento asociado (label), optamos por la opción de utilizar una libreria que nos da una nivel de sentimiento en cada noticia según su polaridad (negativo o positivo).

La idea es usar textBlob y con base en la distribución de la polaridad vamos a asignar un sentimiento a cada noticia: Negativo, neutral o positivo


## Instalar librerias y complementos

In [ ]:
! pip install pandas
! pip install pyspark
! pip install metapy
! pip install boto3
!pip install nltk
!pip install numpy
!pip install re
!pip install codecs
!pip install matplotlib

# Cargar librerias

In [ ]:
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
import metapy
import requests, zipfile, io, os, boto3

import nltk
import pandas as pd
import numpy as np
import re
import codecs

from nltk.corpus import stopwords



nltk.download('punkt')
nltk.download('stopwords')

 
stop_words_nltk = set(stopwords.words('english'))

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id
import re
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.sql.types import StringType

sc = SparkContext('local', "app-topic-detection") 
spark = SparkSession(sc)

# Cargar datos necesarios 

In [ ]:
s3 =  boto3.client('s3', region_name='us-east-1')
with open('data/news/news.dat', 'wb') as f:
    s3.download_fileobj('finaltext','news.dat', f)



obj = s3.get_object(Bucket='finaltext', Key=u'news.csv')
df = pd.read_csv(obj['Body'])

df['all']=df.title + df.content

df2=df[['all']]
df2.to_csv('aux.csv')
df2.head(1000).to_csv('mini.csv')

## Inverted index using metapy


In [ ]:
#!rm -rf news-idx
idx = metapy.index.make_inverted_index('miniconfig.toml')


# IR: Querys

In [ ]:
ranker = metapy.index.OkapiBM25()
query = metapy.index.Document()
query.content('Trump hates china') # query from AP news
top_docs = ranker.score(idx, query, num_results=5)

index=[tup[0] for tup in top_docs]
df.loc[index,['title','content']]


# LDA on spark

### Pre process data

Here we load data to spark and make some preprocessing over the text

In [ ]:
#rawdata=spark.read.csv('aux.csv', inferSchema=True, header=True)

from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer

rawdata = spark.read.load("mini.csv", format="csv", header=True)

rawdata["all"].cast(StringType())




def cleanup_text(record):
    text  = record[0]
    uid   = record[1]
    try:
        words = text.split()
    except:
        words = 'a about'
        
    
    
    
    # Default list of Stopwords
    stopwords_core = ['a', u'about', u'above', u'after', u'again', u'against', u'all', u'am', u'an', u'and', u'any', u'are', u'arent', u'as', u'at', 
    u'be', u'because', u'been', u'before', u'being', u'below', u'between', u'both', u'but', u'by', 
    u'can', 'cant', 'come', u'could', 'couldnt', 
    u'd', u'did', u'didn', u'do', u'does', u'doesnt', u'doing', u'dont', u'down', u'during', 
    u'each', 
    u'few', 'finally', u'for', u'from', u'further', 
    u'had', u'hadnt', u'has', u'hasnt', u'have', u'havent', u'having', u'he', u'her', u'here', u'hers', u'herself', u'him', u'himself', u'his', u'how', 
    u'i', u'if', u'in', u'into', u'is', u'isnt', u'it', u'its', u'itself', 
    u'just', 
    u'll', 
    u'm', u'me', u'might', u'more', u'most', u'must', u'my', u'myself', 
    u'no', u'nor', u'not', u'now', 
    u'o', u'of', u'off', u'on', u'once', u'only', u'or', u'other', u'our', u'ours', u'ourselves', u'out', u'over', u'own', 
    u'r', u're', 
    u's', 'said', u'same', u'she', u'should', u'shouldnt', u'so', u'some', u'such', 
    u't', u'than', u'that', 'thats', u'the', u'their', u'theirs', u'them', u'themselves', u'then', u'there', u'these', u'they', u'this', u'those', u'through', u'to', u'too', 
    u'under', u'until', u'up', 
    u'very', 
    u'was', u'wasnt', u'we', u'were', u'werent', u'what', u'when', u'where', u'which', u'while', u'who', u'whom', u'why', u'will', u'with', u'wont', u'would', 
    u'y', u'you', u'your', u'yours', u'yourself', u'yourselves']
    
    # Custom List of Stopwords - Add your own here
    stopwords_custom = ['']
    stopwords = stopwords_core + stopwords_custom
    stopwords = [word.lower() for word in stopwords]    
    
    text_out = [re.sub('[^a-zA-Z0-9]','',word) for word in words]                                       # Remove special characters
    text_out = [word.lower() for word in text_out if len(word)>2 and word.lower() not in stopwords]     # Remove stopwords and words under X length
    return text_out
        

    return tokens

udf_cleantext = udf(cleanup_text , ArrayType(StringType()))
clean_text = rawdata.withColumn("words", udf_cleantext(struct([rawdata[x] for x in rawdata.columns])))

### Embedings + LDA

here we create the features of each line and then make the LDA itself with k topics

In [ ]:
# Term Frequency Vectorization  - Option 2 (CountVectorizer)    : 
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures", vocabSize = 1000)
cvmodel = cv.fit(clean_text)
featurizedData = cvmodel.transform(clean_text)

vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# Generate 25 Data-Driven Topics:
lda = LDA(k=5, seed=123, optimizer="em", featuresCol="features")

ldamodel = lda.fit(rescaledData)

#model.isDistributed()
#model.vocabSize()

ldatopics = ldamodel.describeTopics()
#ldatopics.show(25)

def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))

### Show topics

In [ ]:
ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(50,False)

### Add detected topic to each line

In [ ]:
ldaResults = ldamodel.transform(rescaledData)

ldaResults.select('all','words','features','topicDistribution').show()

### Add principal topic to each line

In [ ]:
from pyspark.sql.types import IntegerType


def foo(topicDistribution):
    dom = topicDistribution[0]
    index_dom = 0
    for index in range(len(topicDistribution)):
        if (topicDistribution[index]>dom):
            dom=topicDistribution[index]
            index_dom=index
    
    return index_dom

udf_seltop = udf(foo , IntegerType())
aaa = ldaResults.withColumn("topic_prin", udf_seltop(ldaResults.topicDistribution))


aaa.select('all','topic_prin').show()

# Sentiment Analysis